# Reference :
- https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai

# Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
# Important classes to build multi-agent systems using CrewAI
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents, shown in the commented section

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [4]:
"""

# ---
# Some other LLM examples
# ---

# Mistral API
os.environ["OPENAI_API_KEY"]=your-mistral-api-key
os.environ["OPENAI_API_BASE"]=https://api.mistral.ai/v1
os.environ["OPENAI_MODEL_NAME"]="mistral-small"

# Hugging Face (HuggingFaceHub endpoint)
from langchain_community.llms import HuggingFaceHub
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)
# Pass "llm" to your agent function

# Cohere
from langchain_community.chat_models import ChatCohere
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()
# Pass "llm" to your agent function

"""

'\n\n# ---\n# Some other LLM examples\n# ---\n\n# Mistral API\nos.environ["OPENAI_API_KEY"]=your-mistral-api-key\nos.environ["OPENAI_API_BASE"]=https://api.mistral.ai/v1\nos.environ["OPENAI_MODEL_NAME"]="mistral-small"\n\n# Hugging Face (HuggingFaceHub endpoint)\nfrom langchain_community.llms import HuggingFaceHub\nllm = HuggingFaceHub(\n    repo_id="HuggingFaceH4/zephyr-7b-beta",\n    huggingfacehub_api_token="<HF_TOKEN_HERE>",\n    task="text-generation",\n)\n# Pass "llm" to your agent function\n\n# Cohere\nfrom langchain_community.chat_models import ChatCohere\nos.environ["COHERE_API_KEY"] = "your-cohere-api-key"\nllm = ChatCohere()\n# Pass "llm" to your agent function\n\n'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
# Create an instance of Agent class
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [6]:
# Create an instance of Agent class
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [7]:
# Create an editor agent - instance of Agent class
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer 

Final Answer: 

Content Plan: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in today's world

II. Latest Trends in Artificial Intelligence
- Machine Learning advancements
- Natural Language Processing developments
- AI ethics and regulations

III. Key Players in the AI Industry
- Google
- Amazon
- Microsoft
- IBM
- Tesla

IV. Noteworthy News in Artificial Intelligence
- Breakthroughs in AI healthcare application

I now can give a great answer

Final Answer:

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) continues to be at the forefront of technological advancements, shaping the way we interact with machines and transforming various industries. From virtual assistants to self-driving cars, AI has become an integral part of our daily lives, revolutionizing the future of technology. This blog post will delve into the latest trends, key players, and noteworthy news in the AI industry, providing valuable insights for tech enthusiasts, business professionals, students, and anyone interested in the evolving landscape of AI.

## Latest Trends in Artificial Intelligence
The advancements in Machine Learning stand out as one of the most significant trends in AI, allowing machines to learn from data and enhance their performance without explicit programming. Natural Language Processing (NLP) is also making waves, enabling machines to compre

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) continues to be at the forefront of technological advancements, shaping the way we interact with machines and transforming various industries. From virtual assistants to self-driving cars, AI has become an integral part of our daily lives, revolutionizing the future of technology. This blog post will delve into the latest trends, key players, and noteworthy news in the AI industry, providing valuable insights for tech enthusiasts, business professionals, students, and anyone interested in the evolving landscape of AI.

## Latest Trends in Artificial Intelligence
The advancements in Machine Learning stand out as one of the most significant trends in AI, allowing machines to learn from data and enhance their performance without explicit programming. Natural Language Processing (NLP) is also making waves, enabling machines to comprehend and generate human language effectively. Moreover, the spotlight on AI ethics and regulations is growing, emphasizing the importance of responsible AI development and usage, addressing concerns about potential risks and implications.

## Key Players in the AI Industry
Leading tech giants such as Google, Amazon, Microsoft, IBM, and Tesla are at the forefront of AI research and development, driving innovation and pushing boundaries. These key players are investing heavily in AI technologies, ranging from personalized recommendations to the advancement of autonomous vehicles. Their contributions are instrumental in shaping the future of AI and its applications across various sectors.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in AI healthcare applications have demonstrated significant potential in enhancing patient care and diagnosis, showcasing the transformative power of AI in the medical field. Moreover, the progress in AI-powered autonomous vehicles, notably by companies like Tesla, indicates the rapid evolution of self-driving technology. Additionally, AI-driven personal assistants are becoming increasingly sophisticated, offering users valuable support in daily tasks and delivering tailored recommendations.

## Target Audience Analysis
This blog post caters to a diverse audience, including tech enthusiasts, business professionals seeking to integrate AI solutions, students pursuing AI studies, and individuals curious about the future of technology. Whether you are an industry expert or a novice exploring AI concepts, this post aims to provide informative insights into the latest trends and developments in the AI landscape, fostering a deeper understanding of this dynamic field.

## SEO Keywords
- Artificial Intelligence trends
- AI key players
- Latest AI news
- Artificial Intelligence developments
- Future of AI

## Resources
Forbes, TechCrunch, MIT Technology Review, World Economic Forum, and Artificial Intelligence News are valuable sources for staying abreast of the latest AI advancements and insights.

**Call to Action:** Stay informed about the latest AI updates by subscribing to our newsletter for weekly insights and updates.

By adhering to this comprehensive content strategy, we strive to deliver engaging and informative content on Artificial Intelligence, empowering our audience to stay informed, make informed decisions, and remain up-to-date on the latest industry trends.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [14]:
topic = "Large Language Models"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Large Language Models.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: Unveiling the Power of Large Language Models

Introduction:
- Brief explanation of what Large Language Models are and their significance in the field of artificial intelligence.
- Mention of recent advancements and breakthroughs in the development of Large Language Models.
- Preview of key points to be discussed in the article.

Key Points:
1. Latest Trends:
- Analysis of the latest trends in Large Language Models, including the rise of models such as GPT-3, BERT, and T5.
- Exploration of how Large Langua

I now can give a great answer

Final Answer:
# Unveiling the Power of Large Language Models

## Introduction
Large Language Models have revolutionized the field of artificial intelligence by leveraging massive amounts of data to generate human-like text. Recent advancements in this technology, such as GPT-3, BERT, and T5, have garnered significant attention for their ability to generate coherent and contextually relevant text. In this article, we will explore the latest trends in Large Language Models, key players driving their development, noteworthy news and ethical considerations, practical applications, and future implications.

## Latest Trends in Large Language Models
The latest trends in Large Language Models showcase their increasing sophistication and versatility. Models like GPT-3, developed by OpenAI, have demonstrated remarkable capabilities in generating text that mimics human writing. These models are being utilized across various industries, including healthcare for medi

In [15]:
Markdown(result)

# Unveiling the Power of Large Language Models

## Introduction
Large Language Models have revolutionized the field of artificial intelligence by leveraging massive amounts of data to generate human-like text. Recent advancements in this technology, such as GPT-3, BERT, and T5, have garnered significant attention for their ability to generate coherent and contextually relevant text. In this article, we will explore the latest trends in Large Language Models, key players driving their development, noteworthy news and ethical considerations, practical applications, and future implications.

## Latest Trends in Large Language Models
The latest trends in Large Language Models showcase their increasing sophistication and versatility. Models like GPT-3, developed by OpenAI, have demonstrated remarkable capabilities in generating text that mimics human writing. These models are being utilized across various industries, including healthcare for medical record analysis, finance for market analysis, and marketing for content generation. The ability of Large Language Models to process and generate text at scale has opened up new possibilities for automation and efficiency.

## Key Players in the Development of Large Language Models
Key players in the development of Large Language Models, such as OpenAI, Google, and Microsoft, have been at the forefront of innovation in this field. Collaboration and partnerships among these players have accelerated the advancement of Large Language Models, leading to breakthroughs in natural language understanding and text generation. These collaborations have also enabled the sharing of resources and expertise, further driving the development of more sophisticated models with enhanced capabilities.

## Noteworthy News and Ethical Considerations
Recent news articles, research papers, and industry reports have shed light on the ethical considerations and challenges associated with the use of Large Language Models. Issues such as bias in language models, data privacy concerns, and the potential misuse of generated content have sparked important discussions within the AI community. It is crucial for developers and researchers to address these ethical considerations and implement safeguards to ensure the responsible use of Large Language Models.

## Practical Applications and Future Implications
The practical applications of Large Language Models are vast and varied, ranging from content generation to conversational AI and data analysis. As these models continue to evolve and improve, their impact on industries and society as a whole is expected to grow exponentially. Looking ahead, the future implications of Large Language Models include enhanced communication tools, improved decision-making processes, and the potential for groundbreaking discoveries in various fields.

## Call to Action
As we delve deeper into the world of Large Language Models, it is essential for readers to stay informed and engaged with the latest developments in artificial intelligence and machine learning. By exploring further resources and staying updated on the advancements in Large Language Models, we can better understand the potential of this technology and its implications for the future.

In conclusion, Large Language Models represent a significant milestone in the field of artificial intelligence, with the power to transform how we interact with technology and generate content. By staying informed and actively participating in the discourse surrounding Large Language Models, we can shape the responsible development and deployment of this groundbreaking technology.

Remember to stay curious, keep learning, and embrace the possibilities that Large Language Models offer for innovation and creativity.